In [1]:
import submitit
import inspect
import os
import pandas as pd

from frust.stepper import Stepper
import frust.vis as vis
from pathlib import Path
from tooltoad.chemutils import xyz2mol

from rdkit import Chem

In [2]:
def _nuse():
    try:
        if "SLURM_JOB_ID" in os.environ:
            from nuse import start_monitoring
            start_monitoring(filter_cgroup=True)
    except ImportError:
        pass

In [3]:
COMMON_DIR = "run_ts1_test"
COMMON_DIR = Path(COMMON_DIR)
COMMON_DIR.mkdir(exist_ok=True)

f = Path("../structures/ts1.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

smile = Chem.MolToSmiles(mol)

step = Stepper(smile, step_type="mol", save_output_dir=False)
df = step.build_initial_df(mols)
df.loc[0, "smiles"] = smile
df.to_parquet(f"{COMMON_DIR}/init.parquet")

vis.MolTo3DGrid(f)

2025-11-06 12:49:14 INFO  frust.stepper: Working dir: .


[12:49:15] Explicit valence for atom # 10 H, 2, is greater than permitted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
def run_hess(
        parquet_path: str,
        n_cores: int = 1,
        mem_gb: int = 1,
        save_dir: str | None = None,
        work_dir: str | None = None):
        
    _nuse()
    
    df = pd.read_parquet(f"{save_dir}/{parquet_path}")

    ligand_smiles = list(dict.fromkeys(df["smiles"].tolist()))
    step = Stepper(
        ligand_smiles,
        n_cores=n_cores,
        memory_gb=mem_gb,
        output_base=save_dir,
        save_calc_dirs=False,
        save_output_dir=True,
        work_dir=work_dir,
    )

    df = step.orca(df, name="hess", options={
        "wB97X-D3"  : None,
        "6-31G**"   : None,
        "Freq"     : None,
    }, read_files=["input.hess"])

    stem = parquet_path.rsplit('.', 1)[0]
    fn_name = inspect.currentframe().f_code.co_name
    parquet_name = fn_name.split("_")[1]
    out_parquet = stem + f".{parquet_name}.parquet"
    df.to_parquet(f"{save_dir}/{out_parquet}")


#run_hess("init.parquet", save_dir=common_dir)

In [5]:
def run_optts(
        parquet_path: str,
        n_cores: int = 1,
        mem_gb: int = 1,
        save_dir: str | None = None,
        work_dir: str | None = None):

    _nuse()

    df = pd.read_parquet(f"{save_dir}/{parquet_path}")

    ligand_smiles = list(dict.fromkeys(df["smiles"].tolist()))
    step = Stepper(
        ligand_smiles,
        n_cores=n_cores,
        memory_gb=mem_gb,
        output_base=save_dir,
        save_calc_dirs=False,
        save_output_dir=True,
        work_dir=work_dir,
    )

    df = step.orca(df, name="OptTS", options={
        "wB97X-D3"  : None,
        "6-31G**"   : None,
        "TightSCF"  : None,
        "SlowConv"  : None,
        "OptTS"     : None,
        "NoSym"     : None,
    }, use_last_hess=True)

    stem = parquet_path.rsplit('.', 1)[0]
    fn_name = inspect.currentframe().f_code.co_name
    parquet_name = fn_name.split("_")[1]
    out_parquet = stem + f".{parquet_name}.parquet"
    df.to_parquet(f"{save_dir}/{out_parquet}")

#run_optts("init.hess.parquet", save_dir=common_dir)

In [6]:
def run_freq(
        parquet_path: str,
        n_cores: int = 1,
        mem_gb: int = 1,
        save_dir: str | None = None,
        work_dir: str | None = None):
    
    _nuse()

    df = pd.read_parquet(f"{save_dir}/{parquet_path}")

    ligand_smiles = list(dict.fromkeys(df["smiles"].tolist()))
    step = Stepper(
        ligand_smiles,
        n_cores=n_cores,
        memory_gb=mem_gb,
        output_base=save_dir,
        save_calc_dirs=False,
        save_output_dir=True,
        work_dir=work_dir,
    )

    df = step.orca(df, name="freq", options={
        "wB97X-D3"  : None,
        "6-31G**"   : None,
        "Freq"     : None,
    })

    stem = parquet_path.rsplit('.', 1)[0]
    fn_name = inspect.currentframe().f_code.co_name
    parquet_name = fn_name.split("_")[1]
    out_parquet = stem + f".{parquet_name}.parquet"
    df.to_parquet(f"{save_dir}/{out_parquet}")

#run_freq("init.hess.optts.parquet", save_dir=common_dir)

In [7]:
def run_cleanup(save_dir):
    print("[INFO]: Cleanup initiated...")

    """Deletes residual .parquet files"""
    parquet_files = list(Path(save_dir).glob("*.parquet"))
    if not parquet_files:
        print("No parquet files found.")
        return

    if len(parquet_files) < 2:
        print("Passing parquet cleanup, only 1 parquat file found.")
        return
    
    parquet_len = {}
    for f in parquet_files:
        c = len(f.name.split('.'))
        parquet_len[f] = c
        
    parquet_len_sorted = dict(sorted(parquet_len.items(), key=lambda x: x[1], reverse=False))
    dont_delete = max(parquet_len_sorted.values())

    for f, c in parquet_len_sorted.items():
        if c < dont_delete:
            print(f"[INFO]: Removing residual parquet file {f}")
            Path(f).unlink()

    print("[INFO]: Cleanup done!")

#run_cleanup(common_dir)

In [ ]:
df

In [ ]:
RES = {
    "run_hess":   dict(cpus=8, mem=62, time=7200, pre_step="init",),
    "run_optts":   dict(cpus=24, mem=62, time=7200, pre_step="init.hess",),
    "run_freq":  dict(cpus=8, mem=62, time=7200, pre_step="init.hess.optts",),
    "run_cleanup":  dict(cpus=1, mem=1, time=100, pre_step=None),
}

executor = submitit.AutoExecutor(f"logs/{COMMON_DIR}")

last_job, jobs = None, []
for i, j in RES.items():

    fn = globals()[i]

    if last_job:
        extra = {"dependency": f"afterok:{last_job.job_id}",
                 "exclude": "node236,node237,node238,node239"}
    else:
        extra = {}

    executor.update_parameters(
        slurm_job_name=i,
        slurm_partition = "kemi1",
        cpus_per_task=j.get('cpus'),
        mem_gb=j.get('mem'),
        timeout_min=j.get('time'),
        slurm_additional_parameters=extra
    )

    if i != 'run_cleanup':
        job = executor.submit(fn,
                              f"{j.get('pre_step')}.parquet",
                              j.get('cpus'),
                              j.get('mem'),
                              COMMON_DIR,
                              None # work dir
                              )
        last_job = job
    else:
        job = executor.submit(fn, COMMON_DIR) # clean up

    print(f"Submitted: {job.job_id}")

Submitted: 55031833
Submitted: 55031835
Submitted: 55031836
Submitted: 55031837
